# Conditional Gradient Implementation

This simple implemenation is a worked example for a convex function which represents an achiavablility query. The associated diagram (see attachments) validates that each x* is in halfspace currently determined by algorithm 1. 

In [14]:
import numpy as np
from scipy import optimize

In [15]:
t = np.random.uniform(-100., 100, 10)
print(np.where(t >=0, 1, 0))

[0 1 1 0 1 0 1 1 1 1]


In [16]:
w = np.array([0.5, 0.5])
A_test = w[:, np.newaxis].T
print(A_test)

[[0.5 0.5]]


Testing finding a new $s_t$ value. In this test we start with some random $w=(0.5, 0.5)$ which is input into the MOPMC model checker (see attached files) giving the Pareto optimal point of $r_1=(11.7778, 1.336)$. This is the starting point (initial point) to find a new $x^*$ via Frank-Wolfe. Here we use the simplest implementation of Frank-Wolfe and the Achievability query via the ReLU function. 

In this problem because the thresholds are $R_1 <= 12, R_2 <= 1.45$ we are trying to minimise rewards. To minimise rewards in model checking we multiply the rewards by -1 and select the action which maximises the Bellman state equation. 

Similary in optimisation, we are trying to determine if the target sits within the halfspace. For traditional achievability this is the upward closure or downward closure then multiplied by a coefficient of -1. 

In [17]:
def relu_gradient(input):
    return np.where(input >=0, 1, 0)

def f(c0, x):
    return c0 - x

Test the linear programming output (find $s_t$) for one iteration:

In [18]:
Phi = np.array([[11.7778, 1.336]])
x0 = Phi[0]
print(x0.shape)
c0 = np.array([12, 1.45])

# linProg test
c = relu_gradient(f(c0, x0))
print(c, c.shape)
A_ub = -w[:, np.newaxis].T
print(A_ub, A_ub.shape)
b_ub = np.array([np.dot(-w, r) for r in Phi])
print(b_ub, b_ub.shape)

s_t = optimize.linprog(c=c, A_ub=A_ub, b_ub=b_ub)
print(s_t.x)
print(s_t.status)

(2,)
[1 1] (2,)
[[-0.5 -0.5]] (1, 2)
[-6.5569] (1,)
[13.1138  0.    ]
0


In [19]:
# Target check
wt = np.dot(w, np.array([12., 1.45]))
print(np.dot(w, Phi[0]) > wt) # If True need to do point plane projection via 
                              # linear programming to find a new z_* to continue

False


In [20]:
def FrankWolfe(initial_point, iterations, w, Phi, c0):
    x = initial_point

    for iteration in range(1, iterations):
        gamma = 2 / (2 + iterations)
        st = get_st(x, w, Phi, iteration, c0)
        x = x + gamma * (st - x)

    return x

def get_st(x, w, Phi, iter, c0):
    c = relu_gradient(f(c0, x))

    # do a linear programming step which we have to figure out now
    A_ub = -w
    #print(A_ub.shape)
    b_ub = np.array([np.dot(-w[i], r) for i, r in enumerate(Phi)])
    #print(b_ub.shape)
    s_t = optimize.linprog(c=c, A_ub=A_ub, b_ub=b_ub)

    #print("status:", iter, s_t.x)
    return s_t.x
    

### Iteration 1
Running Frank-Wolfe

In [21]:
w = np.array([[0.5, 0.5]])
Phi = np.array([[11.77, 1.336]]) # generated from mopmc framework (alg2)
c0 = np.array([12, 1.45])
initialPoint = Phi[0]

xstar = FrankWolfe(initialPoint, 100, w, Phi, c0) 
print("Final solution:", xstar)


Final solution: [12.91789971  0.18810029]


Computing the new w:

In [22]:
# give c0 an error term to account for numeric error in FW
eps = 0.03
c0eps = c0 + np.array([eps]*c0.shape[0])
nabla_xstar = relu_gradient(f(c0eps,xstar))
print(nabla_xstar)
print("ReLU Grad:", relu_gradient(f(c0eps, xstar)))
denom = np.linalg.norm(relu_gradient(f(c0eps, xstar)), 1)
print(denom)
wnew = nabla_xstar / denom
print("w:",wnew)

[0 1]
ReLU Grad: [0 1]
1.0
w: [0. 1.]


### Iteration 2

In [23]:
W = np.array([[0.5, 0.5], [0., 1.]])
Phi = np.array([[11.7778, 1.336], [14.622, 1.2246]]) # generated from mopmc framework (alg2)

# Target check
wt = np.dot(W[1], np.array([12., 1.45]))
print("Not achievable check:", np.dot(W[1], Phi[1]) > wt)
# If True need to do point plane projection via linear programming to find a new z_* to continue

x0 = Phi[1] # ???? what is the new x0
print(x0.shape)
c0 = np.array([10, 1.45])

# linProg test
c = relu_gradient(f(c0, x0))
print("c", c, c.shape)
A_ub = -W
print("A_ub", A_ub, A_ub.shape)
b_ub = np.array([np.dot(-w, r) for r in Phi])
print("b_ub", b_ub, b_ub.shape)

s_t = optimize.linprog(c=c, A_ub=A_ub, b_ub=b_ub)
print(s_t.x)
print(s_t.status)

Not achievable check: False
(2,)
c [0 1] (2,)
A_ub [[-0.5 -0.5]
 [-0.  -1. ]] (2, 2)
b_ub [[-6.5569]
 [-7.9233]] (2, 1)
[5.1905 7.9233]
0


In [24]:
W = np.array([[0.5, 0.5], [0., 1.]])
Phi = np.array([[11.77, 1.336], [14.622, 1.2246]])
c0 = np.array([10, 1.45])
initialPoint = Phi[1]

xstar = FrankWolfe(initialPoint, 1000, W, Phi, c0) 
print("Final solution:", xstar)

Final solution: [12.25378471  1.2246    ]


In [25]:
# give c0 an error term to account for numeric error in FW
eps = 0.03
c0eps = c0 + np.array([eps]*c0.shape[0])
nabla_xstar = relu_gradient(f(c0eps,xstar))
print(nabla_xstar)
print("ReLU Grad:", relu_gradient(f(c0eps, xstar)))
denom = np.linalg.norm(relu_gradient(f(c0eps, xstar)), 1)
print(denom)
wnew = nabla_xstar / denom
print("w:",wnew)

[0 1]
ReLU Grad: [0 1]
1.0
w: [0. 1.]


The algorithm ends here as there are no new weight vectors and since line 9 of Algorithm 1 has not been violated the threshold is achievable. 